# Résolution de système linéaire par la méthode du pivot de Gauss

## Les étapes du solveur:
* Lecture du fichier en entrée
* Création de la matrice augmentée stockant le système linéaire
* Sélection du pivot (maximum de la diagonale)
* Propagation du pivot sur les lignes concernées
* Résolution du problème linéaire
* Ecriture du résultat finale dans un fichier en sortie
### Fonctionnalités:
* Génération de grandes matrices de tests
* Recherche du pivot maximum en parallèle (OpenMP)
* Propagation du pivot trouvé en parallèle (OpenMP)

## 1. Lecture du fichier en entrée

La lecture du fichier ce fait à partir des outils de la librairie standard du C, grâce aux fonctions de **stdio.h** permettant la lecture d'un fichier. Cette lecture est ici faite élément par élément dans un fichier structuré de la façon suivante:
<br>
format du fichier d'entrée:
* n : nombre d'inconnues
* n lignes d'équations (n + 1 élements par lignes, séparation par un espace)

exemple: <br>
2<br>
2 1 5<br>
4 -6 -20<br>

## 2. Création de la matrice augmentée stockant le système linaire

Nous stockons le système linaire, lu à partir du fichier d'entrée, au sein de la structure suivante:
<br>
```C
/// @brief linear system wrapper structure which contains the system matrix as a 1D array (data) 
///        and a pointer array (storage) to facilitate access to elements inside the linear system matrix
typedef struct linear_system_t {
    int nb_unknowns;
    double * data;
    double ** storage;
} linear_system_t;
```

Cette façon de stocker notre système linéaire nous permet dans un permier temps d'acceder aux dimenssions de la matrice augmentée, à partir du premier membre de la structure **nb_unknowns**. De plus, nous pouvons stocker dans cette même structure à l'intérieur d'un tableau unidimensionnel **data** dans lequel tous les coefficients seront stockées séquentiellement. L'ordonancement de nos indices sous forme matricielle au sein du tableau 1D **data** ce fait à partir du tableau de pointeur **storage** qui nous permet d'acceder au élément de **data** comme un tableau bidimensionnel. Les avantages de ce stockage nous permet d'éviter de faire trop d'appel d'allocation dynamique de mémoire par **malloc**, et par conséquent de faire une économisation mémoire.

## 3. Sélection du pivot (maximum de la diagonale)

La stratégie de sélection du pivot est une étape cruciale dans la résolution par méthode de Gauss. Nous utilisons une méthode de pivot partiel, dans laquel nous sélectionnons la valeur la plus grande sur la diagonale. L'algorithme de propagation, vu plus tard, itère sur les lignes de notre système linéaire de la première à la dernière ligne. Il nous faut alors selectionner un nouveau pivot pour chaque ligne parcourue.<br>
Cette sélection de pivot sur la chaque ligne est décrite dans l'algorithme suivant:

```C
/// @brief Finds the pivot used in the currently provided linear system using a partial pivot strategy
/// @param linear_system the linear system
/// @param current_line
/// @param pivot_line
/// @return the pivot
double select_current_pivot(linear_system_t *linear_system, int current_line, int *pivot_line)
{
    int y;

    int nb_matrix_rows = linear_system->nb_unknowns;

    double pivot = 0;
    double p = linear_system->storage[current_line][current_line]; // pivot in diagonal
    double abs_val;
    double col_val;
    double p_abs;
    for (y = current_line + 1; y < nb_matrix_rows; y++)            // check for max value in same column
    {
        col_val = linear_system->storage[y][current_line];
        abs_val = fabs(col_val); //check if the absolute value of the pivot coefficient to be selected
        p_abs = fabs(p);
        pivot = MAX(p_abs, abs_val);
        if(p_abs != pivot) *pivot_line = y; //if the current pivot is updated, we update the line the pivot is on
        p = (pivot == abs_val) ? col_val : p;
    }

    return p;
}
```

Décrivons plus en détail cette fonction C. Nous sélectionnons d'abord comme valeur pivot le coefficient positionner sur la diagonnale de la ligne traité. Il nous suffit ensuite de regarder les coefficients se trouvant sur la même colonne du pivot actuel, sous celui-ci. Nous effectuons une comparaison (fonction **MAX**) par valeur absolue du pivot actuelle avec les valeurs sous celui-ci, ceci afin de sélectionner à la fin la valeur comme pivot la plus grande possible afin d'assurer une certaine stabilité numérique. Dans le cas où le pivot actuelle à été mise à jour, nous enregistrons alors la ligne sur laquelle ce nouveau coefficient se positionne.

## 4. Propagation du pivot sur les lignes concernées

Comme mentionnée précédemment, l'algorithme de propagation du pivot boucle sur chacune des lignes du système linéaire itérativement. Ce processus se retrouve dans le bloc de suivant, définissant cette algorithme de propagation écrit en C:

```C
void linear_system_propagation(linear_system_t *linear_system)
{
    int curr_line;
    int nb_matrix_rows = linear_system->nb_unknowns;
    double pivot;
    int pivot_line;

    //loop iterativly over each row of the linear system matrix
    for (curr_line = 0; curr_line < nb_matrix_rows; curr_line++)
    {
        pivot_line = curr_line;
        
        // selection du pivot pour chaque ligne de la matrice augmentée du systeme lineaire
        pivot = select_current_pivot(linear_system, curr_line, &pivot_line);

        // changement de ligne pour que le pivot soit sur la diagonale
        if (pivot_line != curr_line)
        {
            swap_linear_system_rows(linear_system, curr_line, pivot_line);
        }

        // pivotage de la matrice
        apply_pivot(linear_system, curr_line);
    }
}
```

Dans cette fonction, nous itérons sur chacune des lignes de notre matrice dans laquelle on nous sélectionnons le pivot, et si on constate que l'on a un coefficient trouvée plus grand dans la colonne du pivot sélectionnée, nous pouvons alors échanger la ligne actuelle avec la ligne qui a un plus grand coefficient. Dès lors, nous pouvons alors effectuer le pivotage de la matrice.

## 5. Résolution du problème linéaire

Avant d'obtenir les solutions potentieles du problème linéaire, nous devons d'abord appliquer le pivot trouver durant les étapes précédentes et le propager sur les lignes sous la ligne du pivot. Cette application du pivot s'effectue à partir de la fonction suivante:
<br>
```C
/// @brief Application de la technique de pivot
/// @param linear_system le système linéaire
/// @param pivot_line la ligne du pivot choisi dans le système linéaire
void apply_pivot(linear_system_t *linear_system, int pivot_line)
{
    int i, j;

    int nb_matrix_rows = linear_system->nb_unknowns;

    double pivot = linear_system->storage[pivot_line][pivot_line];
    //propagation du pivot sur les lignes en dessous
    for (i = pivot_line + 1; i < nb_matrix_rows; i++) 
    {
        //propagation du pivot sur les coefficient de chaque lignes
        for (j = pivot_line + 1; j <= nb_matrix_rows; j++) 
        {
            // A[i][j] = A[i][j] - ( (A[i][pi] / A[pi][pi]) * A[pi][j] )
            linear_system->storage[i][j] = linear_system->storage[i][j] - ((linear_system->storage[i][pivot_line] / pivot) * linear_system->storage[pivot_line][j]);
        }
        linear_system->storage[i][pivot_line] = 0;
    }
}
```

À l'intérieur de l'algorithme, nous avons une double boucle (for) imbriquée parcourant chaque coefficient de la matrice en commençant à la ligne et colonne +1 de la ligne de pivot (le pivot se trouvant sur la diagonale). En outre, ces itérations visent à mettre à jour les coefficients des variables dans chaque ligne sous le pivot. 
<br>
Après avoir mis à jour tous les coefficients dans une ligne, la colonne où se trouve le pivot est remise à zéro dans cette ligne (sauf pour la ligne du pivot) pour éviter les calculs inutiles lors des itérations ultérieures.


---
Dorénavant, nous avons une fonction qui nous permet d'effectuer la triangulation de la matrice donnée en entrée avec un algorithme basique de compléxité O(n^3). La dernière étape étant de résoudre le système linéaire en lui-même afin d'obtenir les solutions de ce dernier.
Cette résolution du problème linéaire et calcul des solutions potentielles ce fait à partir de la fonction suivante:


```C
/// @brief Solves the linear system
/// @param linear_system the given linear system
/// @return the solutions to the linear system inside an array
double *solve_linear_system(linear_system_t *linear_system)
{
    int j = 0;
    double result = 0;

    int nb_matrix_rows = linear_system->nb_unknowns;
    int nb_matrix_cols = nb_matrix_rows + 1;

    // allocate memory for the solutions array, initialized with zeros
    double *solutions = (double *)calloc(linear_system->nb_unknowns, sizeof(double));
    if (solutions == NULL)
    {
        fprintf(stderr, "Out of memory!\n");
        exit(EXIT_FAILURE);
    }

    for (int i = nb_matrix_rows - 1; i >= 0; i--)
    { // commencer à la dernière ligne
        result = 0;
        for (j = i; j < nb_matrix_cols - 1; j++)
        {
            if (i != j)
            { // if the the two aren't the same then an initial solution has been found
                // result += A[i][j] * R[j]
                result += linear_system->storage[i][j] * solutions[j];
            }
        }
        // R[i]=(A[i][dim-1]-result)/A[i][i]
        solutions[i] = (linear_system->storage[i][nb_matrix_cols - 1] - result) / linear_system->storage[i][i];
    }
    return solutions;
}
```

Dans cet algorithme, nous calculons les solutions possibles pour chaque ligne en commençant par la dernière ligne du système linéaire, maintenant triangularisé après application du pivot. Nous itérons alors dans la deuxième boucle (for) pour indice j la partie triangulaire de la matrice pour trouver les valeurs constantes utilisées afin de calculer les autres solutions dans les lignes supérieures. Chaque valeur solution trouvée à la ligne i est stockée individuellement dans un tableau unidimensionnel **solutions**, à l'indice i.

## 6. Ecriture du résultat finale dans un fichier en sortie

L'écriture du fichier ce fait de la même façons que la lecture, à partir des outils de la librairie standard du C, grâce aux fonctions de **stdio.h** permettant la lecture d'un fichier. Cette écriture est ici faite élément par élément dans un fichier structuré de la façon suivante:
<br>
Format du fichier de sortie:
* n : nb d'inconnues
* n lignes triangulées (la partie utile de la matrice résultat, on ne stocke pas le zero)
* n valeurs (résultats des inconnus)

Exemple: <br>
2 <br>
4.000 -6.000 -20.000 <br> 
4.000 15.000 <br>
0.625 3.750 <br>

# 7. Parallélisation de fonctionnalités en OpenMP

## 7.1 Recherche du pivot en parallèle avec OpenMP## 7.1 Recherche du pivot maximum en parallèle (OpenMP)

Pour la recherche du pivot en OpenMP on utilise la bibliothèque **omp.h**. Pour ce faire, il nous a paru judicieux de créer un certain nombre de threads qui vont être réparti sur la colonne et qui vont alors chacun trouver le maximum de leur zone de travail respectif et ainsi, on pourra récupérer la plus grande valeur parmi les maximums locaux trouvée par les threads.
On crée alors une région parallèle à l'aide de la commande :

```c
#pragma omp parallel shared(global_max_pivot, global_pivot_line)
```
À la fin du traitement, les threads pourront alors écrire sur les variables partagées **global_max_pivot** et  **global_pivot_line**, respectivement le plus grand pivot trouvé et la ligne sur laquelle il a été trouvée afin de réaliser la permutation de lignes plus tard. 
<br>
On commence alors par déterminer le nombre de lignes que chaque thread devra traiter. On le calcule de la manière suivante :

```c
int step = (nb_matrix_rows + num_threads - 1)/num_threads;
```
Le nombre de lignes que chaque thread traitera sera égale aux nombres de lignes plus le nombre de threads moins 1, le tout divisé par le nombre de threads afin d'avoir une répartition équitable du nombre de lignes à traiter pour chaque thread.
<br>
À partir de l'identifiant de thread, on va déterminer les lignes sur lesquelles chaque thread effectuera sa tâche. Chaque thread commencera alors à partir de l'indice :

```c
start = (thread_id * step) + (current_line + 1);
```
Cela correspond au numéro du thread que l'on multiplie par le nombre de lignes a traité plus la ligne sur lequel on itère plus un pour ne pas récupérer la valeur du pivot actuelle dans le traitement.
<br>
On définit la limite de la zone de traitement du thread de la manière suivante :

```c
stop = start + step;
if(stop > nb_matrix_rows) stop = nb_matrix_rows;
```
La valeur sera alors égale au point de départ de la zone de traitement plus le nombre de lignes à traiter. Si cette valeur dépasse le nombre actuel de ligne de la matrice, la zone d'arrêt sera définie par le nombre de lignes de la matrice pour éviter un débordement.
<br>
On effectue alors le traitement et on définit une région critique afin de mettre à jour le pivot et la ligne de celui-ci afin d'éviter que tous les threads écrivent en même temps sur la variable.

```c
#pragma omp critical
{
    double abs_max_pivot = fabs(local_max_pivot);
    if(abs_max_pivot > fabs(global_max_pivot)) {
        global_max_pivot = local_max_pivot;
        global_pivot_line = local_pivot_line;
    }
}
```

On peut alors retourner le pivot et la ligne sur lequel le pivot maximum a été trouvé.
<br>
### Observations:
On constate alors après un certain nombre de tests sur des matrices de tailles variables allant de 1024 à 8086, que plus le nombre de threads que l'on va utiliser est grand, plus le temps de recherche de ce pivot va augmenter, dû au fait de la répartition du nombre de lignes pour les threads qui va se retrouver de plus en plus inégalitaire au fur et à mesure de l'avancement de l'algorithme. Le nombre de lignes à traiter va diminuer avec l'avancement de la recherche du pivot sur les dernières lignes.
<br><br>
Dès lors, le rapport entre le nombre de threads utilisés et la taille de la matrice ne justifie plus le coût indirect de l'utilisation de OpenMP. Nous entrons alors dans un paradigme de parallélisme dit "Coarse-grained" (https://en.wikipedia.org/wiki/Granularity_(parallel_computing)). Une parallélisation de type "Fine-grained" (i.e. GPUs) serait alors potentiellement plus apte afin de faire ressortir le plus de parallélisme.

In [ ]:
/// @brief Finds the pivot used in the currently provided linear system using a partial pivot strategy
/// @param linear_system the linear system
/// @param current_line
/// @param pivot_line
/// @return the pivot
double omp_select_current_pivot(linear_system_t *linear_system, int current_line, int *pivot_line)
{
    int nb_matrix_rows = linear_system->nb_unknowns;

    int global_pivot_line = *pivot_line;
    double global_max_pivot = linear_system->storage[current_line][current_line];
    #pragma omp parallel shared(global_max_pivot, global_pivot_line)
    {
        int start, stop;
        int thread_id = omp_get_thread_num();
        double local_max_pivot = global_max_pivot;
        int local_pivot_line = global_pivot_line;

        int num_threads = omp_get_num_threads();
        int step = (nb_matrix_rows + num_threads - 1)/num_threads;

        start = (thread_id * step) + (current_line + 1);
        stop = start + step;
        if(stop > nb_matrix_rows) stop = nb_matrix_rows;

        double abs_val;
        double col_val;
        double p_abs;
        //#pragma omp for schedule(static, step) this creates further problems if the step can't be divided evenly
        for (int y = start; y < stop; y++)
        {
            col_val = linear_system->storage[y][current_line];
            abs_val = fabs(col_val); // check if the absolute value of the pivot coefficient to be selected
            p_abs = fabs(local_max_pivot);
            if(abs_val > p_abs) {
                local_pivot_line = y;
                local_max_pivot = col_val;
            }
        }

        #pragma omp critical
        {
            double abs_max_pivot = fabs(local_max_pivot);
            if(abs_max_pivot > fabs(global_max_pivot)) {
                global_max_pivot = local_max_pivot;
                global_pivot_line = local_pivot_line;
            }
        }
    }
    *pivot_line = global_pivot_line;
    return global_max_pivot;
}

## 7.2 Propagation du pivot trouvé en parallèle (OpenMP)

De même que pour la recherche du pivot maximum, nous avons réécrit une nouvelle version de la propagation du pivot sur les autres lignes en une version OpenMP dans le fichier **omp_utils.c**, dûment nommée **omp_apply_pivot**.
La parallélisation de cette fonction s'effectue très facilement du fait qu'il n'y a pas de dépendances apparentes entre les différentes lignes sur lesquelles le pivot est appliqué.
<br><br>
Du fait que dans notre implémentation de la propagation du pivot, les boucles ne sont pas parfaitement imbriquées, nous devons alors faire une parallélisation séparée de chaque boucle (on ne peut donc pas utiliser la directive OpenMP **collapse**).
Nous partageons alors l'itération de la boucle (for) externe, sur chacune des lignes de la matrice augmentée, entre nos threads OpenMP (par la directive suivante :
```C 
#pragma omp for schedule(dynamic, nb_matrix_rows) 
```
)
<br>
De plus, afin d'améliorer les performances de l'application du pivot sur les coefficients de chaque ligne, nous vectorisons explicitement les calculs de coefficients dans la boucle (for) interne pour j. Cette opération ce fait dans la partie suivante de la fonction :
```C
// propagation du pivot sur les coefficient de chaque lignes
#pragma omp simd
for (j = pivot_line + 1; j <= nb_matrix_rows; j++)
{
    linear_system->storage[i][j] -= ((linear_system->storage[i][pivot_line] / pivot) * linear_system->storage[pivot_line][j]);
}
```
<br>
Le reste de la fonction s'éxecute de la même manière que la version séquentielle.

In [ ]:
/// @brief Application de la technique de pivot en OpenMP
/// @param linear_system le système linéaire
/// @param pivot_line la ligne du pivot choisi dans le système linéaire
void omp_apply_pivot(linear_system_t *linear_system, int pivot_line)
{
    int i, j;

    int nb_matrix_rows = linear_system->nb_unknowns;

    double pivot = linear_system->storage[pivot_line][pivot_line];

    #pragma omp parallel private(i)
    {
        // propagation du pivot sur les lignes en dessous
        #pragma omp for schedule(dynamic, nb_matrix_rows)
        for (i = pivot_line + 1; i < nb_matrix_rows; i++)
        {
            // propagation du pivot sur les coefficient de chaque lignes
            #pragma omp simd
            for (j = pivot_line + 1; j <= nb_matrix_rows; j++)
            {
                linear_system->storage[i][j] -= ((linear_system->storage[i][pivot_line] / pivot) * linear_system->storage[pivot_line][j]);
            }
            linear_system->storage[i][pivot_line] = 0;
        }
    }
}

## Benchmark ##
Nous avons effectuer par la suite un benchmark à partir du script **bench_avg_runtime**, sur ROMEO, afin d'examiner le temps d'exécution en moyenne (sur 10) de la version séquentielle et la version parallèle (OpenMP) avec un nombre de threads évoluant. Ceci a été effectuer sur un fichier de système linéaire contenant 2048 inconnus.
<br>
Les données collécter sous formes de graphes sont les suivantes: 
<br><br>
![benchmark graph with both sequential and paralle output](./img/benchmark_matrix2048_16_threads.png)
<br>
Nous observons que la version séquentielle s'éxécute entre 13 et 13.2 secondes. On constate alors que la version OpenMP nous avons un gain de temps lorsque l'on utilise un nombre de threads inférieur à 3 .
<br>
On peut alors supposer que lorsque l'on traitera des matrice plus grande; on aura un rapport de l'utilisation de threads plus avantageux que dans cette exemple, avec de meilleurs temps.

# Execution du script de résolution de système linaire

![benchmark graph with seq](./img/new_ev_seq.png)

# Parallélisation sur GPU avec CUDA

* Séléction du pivot de la colonne actuelle par réduction CUDA
* Propagation du pivot en parallèle

## Séléction du pivot de la colonne actuelle par réduction CUDA

Comme le nom l'indique, la séléction du pivot partiel est en soit une opération séquentielle du fait que les valeurs du système linéaire change pour chaque propagation, pour chaque ligne sur lesquels nous itérons.

De ce fait la séléction du pivot ne peut ce faire que sur la colonne actuelle. Dès lors, l'utilisation de CUDA ici serait dans le cadre d'une réduction entre nos threads afin de répartir la tâche de recherche du pivot maximum dans notre colonne. Les threads sont répartis sous la diagonale et recherche chacun un maximum local. Dès que chaque thread à effectuer sa recherche de maximum local, il regarde les valeurs de chacun des autres threads dans la mémoire partagée et cherche alors le maximum global.



In [ ]:
__global__ void find_pivot_and_swap(double *d_linear_system, int n_rows, int n_cols, int current_line, int *d_pivot_line)
{
    extern __shared__ double shared_data[];

    int tid = threadIdx.x;
    int col = current_line;
    double *local_max = shared_data;
    int *local_pivot = (int *)&local_max[blockDim.x];

    // Step 1: Identify the pivot row
    double pivot = 0;
    double p = d_linear_system[current_line * n_cols + col];
    double abs_val;
    double col_val;
    double p_abs;
    int pivot_row = current_line;
    for (int row = tid + current_line + 1; row < n_rows; row += blockDim.x)
    {
        col_val = d_linear_system[row * n_cols + col];
        abs_val = fabs(col_val); // check if the absolute value of the pivot coefficient to be selected
        p_abs = fabs(p);
        pivot = MAX(p_abs, abs_val);
        if (p_abs != pivot)
            pivot_row = row;
        p = (pivot == abs_val) ? col_val : p;
    }

    __syncthreads();

    local_max[tid] = p;
    local_pivot[tid] = pivot_row;

    __syncthreads();

    // Reduce to find the maximum value and corresponding row index
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)
    {
        if (tid < stride)
        {
            if (fabs(local_max[tid]) < fabs(local_max[tid + stride]))
            {
                local_max[tid] = local_max[tid + stride];
                local_pivot[tid] = local_pivot[tid + stride];
            }
        }
        __syncthreads();
    }

    __syncthreads();

    if (tid == 0)
    {
        *d_pivot_line = local_pivot[0];
    }
}

## Propagation du pivot en parallèle

Dans le propagation du pivot, il s'agit aussi d'une opération séquentielle où chaque opération sur une ligne est indépendant des autres lignes, il est alors pertinent de paralléliser ce procédé. Chaque thread est associée à un élément de la matrice. On stocke alors dans la mémoire partagée la ligne du pivot, et si le thread correspond à un élément de la ligne de pivot, l'élément est alors chargée dans la mémoire partagée. De plus, si un thread correspond à un élément dans une ligne sous la ligne du pivot, l'élimination de Gauss est effectuée pour cet élément. Cela implique de soustraire un multiple de la ligne du pivot de la ligne actuelle pour obtenir un zéro dans la position du pivot.



In [ ]:
// paralell selection of the max absolute value gaussian pivot
template <int BLOCK_SIZE>
__global__ void gauss_elimination(double *d_linear_system, int n_rows, int n_cols, int pivot_line)
{
    // Thread ID
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;

    // Allocate shared memory for the pivot row
    extern __shared__ double pivot_row[];

    if (col < n_cols)
    {

        // Load the pivot row into shared memory
        if (row == pivot_line)
        {
            pivot_row[col] = d_linear_system[pivot_line * n_cols + col];
        }
        __syncthreads();

        // Perform Gaussian elimination for elements in rows below the pivot line
        if (row > pivot_line && row < n_rows)
        {
            double pivot_value = pivot_row[pivot_line];

            double factor = d_linear_system[row * n_cols + pivot_line] / pivot_value;

            if (col >= pivot_line)
            {
                d_linear_system[row * n_cols + col] -= factor * pivot_row[col];
            }
        }
        __syncthreads();
    }
}

### Problème actuel

Problème de selection de pivot en parallèle qui ne trouve pas parfois la bonne ligne maximum de pivot et qui donc fait un échange de ligne incorrect. Les résulats finaux sont alors faussées. De plus, travaillant sur un petit GPU nous manquons de mémoire partagée ainsi que de nombre de threads afin d'effectuer la recherche de pivot ainsi que l'elimination de façons efficace pour de plus grande matrice. 

# Quelques résultats

|   Taille des matrices   |   Sequentielle |   Parallèle (8 Threads) | CUDA |
|---    |:-:    |:-:    |--:    |
|   2   |   0.000001     |   0.004984 |   0.000508 |
|   4   |   0.000001     |   0.004063 |   0.000699 |
|   8   |   0.000004     |   0.012294 |   0.000943 |
|   16   |   0.000016    |   0.000472 |   0.002042 |
|   32   |   0.000068    |   0.000318 |   0.003801 |
|   64   |   0.000597    |   0.002587 |   0.009965 |
|   128   |   0.003394   |   0.011277 |   0.030781 |
|   256   |   0.030089   |   0.058987 |   0.232541 |
|   512   |   0.212622   |   0.352585 |   2.264164 |